In [1]:
%pip install langchain

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.


## Import LangChain related modules

In [2]:
import openai
import os

os.environ['OPENAI_API_KEY'] = '*'

os.environ["http_proxy"]="127.0.0.1:7890"
os.environ["https_proxy"]="127.0.0.1:7890"

In [16]:
from langchain import PromptTemplate
import re
import tenacity
from typing import List, Dict, Callable
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import RegexParser
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage,
    BaseMessage,
)
from langchain.memory import ChatMessageHistory

## Define classes

In [4]:
class DialogueAgent:
    def __init__(
        self,
        name: str,
        system_message: SystemMessage,
        model: ChatOpenAI,
    ) -> None:
        self.name = name
        self.system_message = system_message
        self.model = model
        self.prefix = f"{self.name}: "
        self.reset()
        
    def reset(self):
        self.message_history = ["Here is the conversation so far."]

    def send(self) -> str:
        """
        Applies the chatmodel to the message history
        and returns the message string
        """
        message = self.model(
            [
                self.system_message,
                HumanMessage(content="\n".join(self.message_history + [self.prefix])),
            ]
        )
        return message.content

    def receive(self, name: str, message: str) -> None:
        """
        Concatenates {message} spoken by {name} into message history
        """
        self.message_history.append(f"{name}: {message}")


class DialogueSimulator:
    def __init__(
        self,
        agents: List[DialogueAgent],
        selection_function: Callable[[int, List[DialogueAgent]], int],
    ) -> None:
        self.agents = agents
        self._step = 0
        self.select_next_speaker = selection_function
        
    def reset(self):
        for agent in self.agents:
            agent.reset()

    def inject(self, name: str, message: str):
        """
        Initiates the conversation with a {message} from {name}
        """
        for agent in self.agents:
            agent.receive(name, message)

        # increment time
        self._step += 1

    def step(self) -> tuple[str, str]:
        # 1. choose the next speaker
        speaker_idx = self.select_next_speaker(self._step, self.agents)
        speaker = self.agents[speaker_idx]

        # 2. next speaker sends message
        message = speaker.send()

        # 3. everyone receives message
        for receiver in self.agents:
            receiver.receive(speaker.name, message)

        # 4. increment time
        self._step += 1

        return speaker.name, message

In [5]:
class BiddingDialogueAgent(DialogueAgent):
    def __init__(
        self,
        name,
        system_message: SystemMessage,
        bidding_template: PromptTemplate,
        model: ChatOpenAI,
    ) -> None:
        super().__init__(name, system_message, model)
        self.bidding_template = bidding_template
        
    def bid(self) -> str:
        """
        Asks the chat model to output a bid to speak
        """
        prompt = PromptTemplate(
            input_variables=['message_history', 'recent_message'],
            template = self.bidding_template
        ).format(
            message_history='\n'.join(self.message_history),
            recent_message=self.message_history[-1])
        bid_string = self.model([SystemMessage(content=prompt)]).content
        return bid_string
        

## Define participants and debate topic

In [6]:
case_description = """On September 17, 2012, Henan Henghe Real Estate Co., Ltd. ("Henghe Company") and Zhongtian Construction Group Co., Ltd. ("Zhongtian Company") signed a Contract for Construction of the Construction Project of the Henghe International Business Convention and Exhibition Center, under which Zhongtian Company would construct the project involved in the case. On June 25, 2013, Henghe Company issued a Bid Winning Notice to Zhongtian Company, notifying Zhongtian Company that it succeeded in the bidding for the Henghe International Business Convention and Exhibition Center Project on Kaiyuan Avenue, Luolong District, Luoyang City. On June 26, 2013, Henghe Company and Zhongtian Company signed a Contract for Construction of Construction Project, in which the both parties agreed on the construction term, project cost, liability for breach of contract, and other matters related to the project. After the contract was signed, Zhongtian Company entered the site for construction. During the construction, as Henghe Company defaulted on project cost payment, Zhongtian Company delivered letters to Henghe Company on November 12, 2013, November 26, 2013, and December 23, 2014, claiming that Henghe Company should immediately pay project cost in arrears, and should pay liquidated damages and be liable for the corresponding loss as agreed in the contract. In April and May 2014, Henghe Company signed a Construction Project Cost Consultation Contract with TAHP Project Management (Beijing) Co., Ltd. ("TAHP Company") and retained TAHP Company to conduct a settlement examination of the project involved in the case. On November 3, 2014, TAHP Company issued a Report on the Settlement Examination of Henghe International Business Exhibition Center. Henghe Company, Zhongtian Company, and TAHP Company affixed their official seals to, and signed for confirmation, the examination summary schedule in the examination report. On November 24, 2014, Zhongtian Company received a notice that the Intermediate People's Court of Jiaozuo City, Henan Province would have the project involved in the case auctioned as per the application by another creditor of Henghe Company. On December 1, 2014, the Ninth Construction Company of Zhongtian Company submitted to the Intermediate People's Court of Jiaozuo City, Henan Province a Letter Regarding the Auction of the Henghe International Business Convention and Exhibition Center Project under Construction, stating that Zhongtian Company, as the contractor of the Henghe International Business Convention and Exhibition Center Project under construction, had completed construction works worth 287 million yuan since the commencement of construction of the project, and requested the confirmation of its priority of payment and participation in the auction throughout the process. Zhongtian Company and Henghe Company both admitted that the project involved in the case were suspended on February 5, 2015."""
character_names = ['Henan Henghe Real Estate Co., Ltd.', 'Zhongtian Construction Group Co., Ltd.', 'Mediator']
word_limit = 50

In [7]:
game_description = f"""Here is the conversation happens in the alternative dispute resolution (ADR) process.\
The case description: {case_description}.\
The conversation includes the mediator and the dispute parties: {', '.join(character_names)}."""

player_descriptor_system_message = SystemMessage(
    content="You can add detail to the task or claim of each participant.")

def generate_character_description(character_name):
    character_specifier_prompt = [
        player_descriptor_system_message,
        HumanMessage(content=
            f"""{game_description}
            For the {character_name} character, please reply with his claims or tasks in this ADR process.\
            Reply in {word_limit} words or less, be as specific as possible. 
            Speak directly to {character_name}.
            Do not add anything else."""
            )
    ]
    character_claim = ChatOpenAI(temperature=1.0)(character_specifier_prompt).content
    return character_claim

def generate_character_header(character_name, character_claim):
    return f"""{game_description}
Your are the {character_name} in this case.
Your claim (task) is as follows: {character_claim}
You are debating with other party in this ADR conversation.
Your goal is to debate and negotiate to reach a agreement.
"""

def generate_character_system_message(character_name, character_header):
    return SystemMessage(content=(
    f"""{character_header}
You will speak in the style of {character_name}, and follow their role in this case.
The mediator will be hosting this conversation.
You will present claim and supportive evidence during the debate.
Do not say the same things over and over again.
Speak in the first person from the perspective of {character_name}
For describing your own body movements, wrap your description in '*'.
Do not change roles!
Do not speak from the perspective of anyone else.
Speak only from the perspective of {character_name}.
Stop speaking the moment you finish speaking from your perspective.
Never forget to keep your response to {word_limit} words!
Do not add anything else.
    """
    ))

character_descriptions = [generate_character_description(character_name) for character_name in character_names]
character_headers = [generate_character_header(character_name, character_description) for character_name, character_description in zip(character_names, character_descriptions)]
character_system_messages = [generate_character_system_message(character_name, character_headers) for character_name, character_headers in zip(character_names, character_headers)]
                                                                                                                                       

In [8]:
for character_name, character_description, character_header, character_system_message in zip(character_names, character_descriptions, character_headers, character_system_messages):
    print(f'\n\n{character_name} Description:')
    print(f'\n{character_description}')
    print(f'\n{character_header}')
    print(f'\n{character_system_message.content}')



Henan Henghe Real Estate Co., Ltd. Description:

Henan Henghe Real Estate Co., Ltd.'s task in this ADR process is to negotiate with Zhongtian Construction Group Co., Ltd. and come to a settlement regarding the arrears of payment for the construction project, as well as the liquidated damages and corresponding loss as stated in the contract. They may also need to discuss their involvement in the auction process and potential confirmation of priority of payment.

Here is the conversation happens in the alternative dispute resolution (ADR) process.The case description: On September 17, 2012, Henan Henghe Real Estate Co., Ltd. ("Henghe Company") and Zhongtian Construction Group Co., Ltd. ("Zhongtian Company") signed a Contract for Construction of the Construction Project of the Henghe International Business Convention and Exhibition Center, under which Zhongtian Company would construct the project involved in the case. On June 25, 2013, Henghe Company issued a Bid Winning Notice to Zhong

## Output parser for bids

In [9]:
class BidOutputParser(RegexParser):
    def get_format_instructions(self) -> str:
        return 'Your response should be an integer delimited by angled brackets, like this: <int>.'     
    
bid_parser = BidOutputParser(
    regex=r'<(\d+)>', 
    output_keys=['bid'],
    default_output_key='bid')

## Generate bidding system message

In [10]:
def generate_character_bidding_template(character_header):
    bidding_template = (
    f"""{character_header}

```
{{message_history}}
```

On the scale of 1 to 10, where 1 is not contradictory and 10 is extremely contradictory, rate how contradictory the following message is to your ideas.

```
{{recent_message}}
```

{bid_parser.get_format_instructions()}
Do nothing else.
    """)
    return bidding_template

character_bidding_templates = [generate_character_bidding_template(character_header) for character_header in character_headers]
                                      

In [11]:
for character_name, bidding_template in zip(character_names, character_bidding_templates):
    print(f'{character_name} Bidding Template:')
    print(bidding_template)

Henan Henghe Real Estate Co., Ltd. Bidding Template:
Here is the conversation happens in the alternative dispute resolution (ADR) process.The case description: On September 17, 2012, Henan Henghe Real Estate Co., Ltd. ("Henghe Company") and Zhongtian Construction Group Co., Ltd. ("Zhongtian Company") signed a Contract for Construction of the Construction Project of the Henghe International Business Convention and Exhibition Center, under which Zhongtian Company would construct the project involved in the case. On June 25, 2013, Henghe Company issued a Bid Winning Notice to Zhongtian Company, notifying Zhongtian Company that it succeeded in the bidding for the Henghe International Business Convention and Exhibition Center Project on Kaiyuan Avenue, Luolong District, Luoyang City. On June 26, 2013, Henghe Company and Zhongtian Company signed a Contract for Construction of Construction Project, in which the both parties agreed on the construction term, project cost, liability for breach o

## Use an LLM to create an elaborate on debate topic

In [12]:
# topic_specifier_prompt = [
#     SystemMessage(content="You can make a task more specific."),
#     HumanMessage(content=
#         f"""{game_description}
        
#         You are the debate moderator.
#         Please make the debate topic more specific. 
#         Frame the debate topic as a problem to be solved.
#         Be creative and imaginative.
#         Please reply with the specified topic in {word_limit} words or less. 
#         Speak directly to the presidential candidates: {*character_names,}.
#         Do not add anything else."""
#         )
# ]
# specified_topic = ChatOpenAI(temperature=1.0)(topic_specifier_prompt).content

# print(f"Original topic:\n{topic}\n")
# print(f"Detailed topic:\n{specified_topic}\n")

Original topic:
transcontinental high speed rail

Detailed topic:
Candidates, the transcontinental high speed rail has been planned with a budget of $100 billion. However, the right-of-way acquisition process is facing legal and financial obstacles. How would you ensure that the project stays within budget while maintaining its proposed route?



## Define the speaker selection function

In [12]:
@tenacity.retry(stop=tenacity.stop_after_attempt(2),
                    wait=tenacity.wait_none(),  # No waiting time between retries
                    retry=tenacity.retry_if_exception_type(ValueError),
                    before_sleep=lambda retry_state: print(f"ValueError occurred: {retry_state.outcome.exception()}, retrying..."),
                    retry_error_callback=lambda retry_state: 0) # Default value when all retries are exhausted
def ask_for_bid(agent) -> str:
    """
    Ask for agent bid and parses the bid into the correct format.
    """
    bid_string = agent.bid()
    bid = int(bid_parser.parse(bid_string)['bid'])
    return bid

In [13]:
import numpy as np

def select_next_speaker(step: int, agents: List[DialogueAgent]) -> int:
    bids = []
    for agent in agents:
        bid = ask_for_bid(agent)
        bids.append(bid)
        
    # randomly select among multiple agents with the same bid
    max_value = np.max(bids)
    max_indices = np.where(bids == max_value)[0]
    idx = np.random.choice(max_indices)
    
    print('Bids:')
    for i, (bid, agent) in enumerate(zip(bids, agents)):
        print(f'\t{agent.name} bid: {bid}')
        if i == idx:
            selected_name = agent.name
    print(f'Selected: {selected_name}')
    print('\n')
    return idx

## Main loop

In [14]:
characters = []
for character_name, character_system_message, bidding_template in zip(character_names, character_system_messages, character_bidding_templates):
    characters.append(BiddingDialogueAgent(
        name=character_name,
        system_message=character_system_message,
        model=ChatOpenAI(temperature=0.2),
        bidding_template=bidding_template,
    ))

In [15]:
max_iters = 10
n = 0

simulator = DialogueSimulator(
    agents=characters,
    selection_function=select_next_speaker
)
simulator.reset()
# simulator.inject('Debate Moderator', specified_topic)
# print(f"(Debate Moderator): {specified_topic}")
print('\n')

while n < max_iters:
    name, message = simulator.step()
    print(f"({name}): {message}")
    print('\n')
    n += 1



Bids:
	Henan Henghe Real Estate Co., Ltd. bid: 2
	Zhongtian Construction Group Co., Ltd. bid: 7
	Mediator bid: 1
Selected: Zhongtian Construction Group Co., Ltd.


(Zhongtian Construction Group Co., Ltd.): As the contractor of the Henghe International Business Convention and Exhibition Center Project under construction, we have completed construction works worth 287 million yuan since the commencement of the project. We request confirmation of our priority of payment and participation in the auction process. We have evidence to support our claim and are willing to present it.


Bids:
	Henan Henghe Real Estate Co., Ltd. bid: 6
	Zhongtian Construction Group Co., Ltd. bid: 1
	Mediator bid: 2
Selected: Henan Henghe Real Estate Co., Ltd.


(Henan Henghe Real Estate Co., Ltd.): I understand your request for confirmation of priority of payment and participation in the auction process. However, we also have concerns regarding the arrears of payment for the construction project, as well as th

In [17]:
history = ChatMessageHistory()

history.messages

[]

In [ ]:
def case_judgement():
    case_judgement_prompt = [
        SystemMessage(content=
            f"""{game_description}
            The parties in this case agreed to solve the dispute through ADR process.\
            This is the conversation during the debate and negotiation: {message history}
            """
            )
    ]
    character_claim = ChatOpenAI(temperature=1.0)(case_judgement_prompt).content
    return character_claim